# SQL Queries in Python

In [ ]:
import sqlite3
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# Function to execute SQL queries
def run_query(query):
    """Executes SQL query and returns results as DataFrame"""
    engine = create_engine("sqlite:///adhoc_requests.db")
    conn = engine.connect()
    df = pd.read_sql(query, conn)
    conn.close()
    return df

In [ ]:
# Query 1: Markets where "Atliq Exclusive" operates in APAC
query1 = """
SELECT DISTINCT market
FROM dim_customer
WHERE customer = 'Atliq Exclusive' AND region ='APAC';
"""
print("Query 1 Results:")
display(run_query(query1))

In [ ]:
# Query 2: Percentage of unique product increase in 2021 vs. 2020
query2 = """
WITH cte1 AS (
    SELECT COUNT(DISTINCT product_code) AS unique_products_2021 
    FROM fact_sales_monthly
    WHERE fiscal_year = 2021
),
cte2 AS (
    SELECT COUNT(DISTINCT product_code) AS unique_products_2020 
    FROM fact_sales_monthly
    WHERE fiscal_year = 2020
)
SELECT *, ROUND(((unique_products_2021 - unique_products_2020) / unique_products_2020) * 100, 2) AS percentage_chg 
FROM cte1 CROSS JOIN cte2;
"""
print("Query 2 Results:")
display(run_query(query2))

In [ ]:
# Query 3: Unique product counts per segment
query3 = """
SELECT segment, COUNT(DISTINCT product_code) AS product_count
FROM dim_product
GROUP BY segment
ORDER BY product_count DESC;
"""
print("Query 3 Results:")
display(run_query(query3))

In [ ]:
# Query 4: Segment with most increase in unique products (2021 vs 2020)
query4 = """
SELECT p.segment,
    COUNT(DISTINCT CASE WHEN S.fiscal_year = 2020 THEN S.product_code END) AS product_count_2020, 
    COUNT(DISTINCT CASE WHEN S.fiscal_year = 2021 THEN S.product_code END) AS product_count_2021,
    (COUNT(DISTINCT CASE WHEN S.fiscal_year = 2021 THEN S.product_code END) -
     COUNT(DISTINCT CASE WHEN S.fiscal_year = 2020 THEN S.product_code END)) AS difference
FROM dim_product AS p
LEFT JOIN fact_sales_monthly AS S ON p.product_code = S.product_code
WHERE S.fiscal_year IN (2020, 2021)
GROUP BY p.segment
ORDER BY difference DESC;
"""
print("Query 4 Results:")
display(run_query(query4))

In [ ]:
# Query 5: Products with highest and lowest manufacturing costs
query5 = """
SELECT p.product_code, p.product, f.manufacturing_cost
FROM dim_product AS p
LEFT JOIN fact_manufacturing_cost AS f ON p.product_code = f.product_code
WHERE manufacturing_cost = (SELECT MAX(manufacturing_cost) FROM fact_manufacturing_cost)
UNION
SELECT p.product_code, p.product, f.manufacturing_cost
FROM dim_product AS p
LEFT JOIN fact_manufacturing_cost AS f ON p.product_code = f.product_code
WHERE manufacturing_cost = (SELECT MIN(manufacturing_cost) FROM fact_manufacturing_cost);
"""
print("Query 5 Results:")
display(run_query(query5))

In [ ]:
# Query 6: Top 5 customers with highest avg pre-invoice discount in India (2021)
query6 = """
WITH cte AS (
    SELECT a.customer_code, a.customer, b.fiscal_year, b.pre_invoice_discount_pct
    FROM dim_customer AS a
    JOIN fact_pre_invoice_deductions AS b
    ON a.customer_code = b.customer_code
    WHERE b.fiscal_year = 2021 AND a.market = 'India'
)
SELECT customer_code, customer, ROUND(AVG(pre_invoice_discount_pct) * 100, 2) AS average_discount_pct
FROM cte
GROUP BY customer_code, customer
ORDER BY average_discount_pct DESC
LIMIT 5;
"""
print("Query 6 Results:")
display(run_query(query6))

In [ ]:
# Query 7: Monthly Gross Sales Amount for "Atliq Exclusive"
query7 = """
WITH cte AS (
    SELECT a.customer, b.date, b.sold_quantity, c.gross_price 
    FROM dim_customer AS a
    JOIN fact_sales_monthly AS b ON a.customer_code = b.customer_code
    JOIN fact_gross_price AS c ON b.product_code = c.product_code
    WHERE a.customer = 'Atliq Exclusive'
)
SELECT strftime('%m', date) AS Month, strftime('%Y', date) AS Year,
       ROUND(SUM(sold_quantity * gross_price) / 1000000, 2) AS gross_sales_amt
FROM cte
GROUP BY Month, Year;
"""
print("Query 7 Results:")
display(run_query(query7))

In [ ]:
# Query 8: Quarter with maximum total sold quantity in 2020
query8 = """
SELECT CASE
         WHEN date BETWEEN '2019-09-01' AND '2019-11-30' THEN 1
         WHEN date BETWEEN '2019-12-01' AND '2020-02-29' THEN 2
         WHEN date BETWEEN '2020-03-01' AND '2020-05-31' THEN 3
         WHEN date BETWEEN '2020-06-01' AND '2020-08-31' THEN 4
       END AS Quarter,
       SUM(sold_quantity) AS total_sold_quantity
FROM fact_sales_monthly
WHERE fiscal_year = 2020
GROUP BY Quarter
ORDER BY total_sold_quantity DESC;
"""
print("Query 8 Results:")
display(run_query(query8))

In [ ]:
# Query 9: Channel with highest gross sales in 2021
query9 = """
WITH cte AS (
    SELECT a.channel, SUM(b.sold_quantity * c.gross_price) / 1000000 AS gross_sales_mln
    FROM dim_customer AS a
    JOIN fact_sales_monthly AS b ON a.customer_code = b.customer_code
    JOIN fact_gross_price AS c ON b.product_code = c.product_code
    WHERE b.fiscal_year = 2021
    GROUP BY a.channel
)
SELECT channel, gross_sales_mln, ROUND((gross_sales_mln / SUM(gross_sales_mln) OVER()) * 100, 2) AS pct_contrib
FROM cte
ORDER BY gross_sales_mln DESC;
"""
print("Query 9 Results:")
display(run_query(query9))

In [ ]:
# Query 10: Top 3 products in each divisionin 2021
query10 = """

WITH cte1 AS (
SELECT 
         a.division,
        a.product_code,
        a.product,
        sum(b.sold_quantity) AS total_sold_quantity
FROM dim_product AS a
JOIN fact_sales_monthly AS b
ON a.product_code = b.product_code
    WHERE b.fiscal_year = 2021
    GROUP BY a.division, a.product_code, a.product
),

-- I need to show 3 top products per division, top products will be based on total sold quantity

cte2 AS (
SELECT *,
rank() OVER (PARTITION BY division ORDER BY total_sold_quantity DESC) as `rank`
        FROM cte1
) 

SELECT * FROM cte2
WHERE `rank` <= 3;
"""

print("Query 9 Results:")
display(run_query(query9))